# 📊 QEPC NBA Backtest Notebook

This notebook measures how well the QEPC NBA engine is performing over time.

It focuses on:
- 🗓️ Defining a backtest date range
- 🧮 Running `run_season_backtest(...)` to simulate all games in that range
- 📈 Summarizing accuracy & spread error
- 🧨 Inspecting the worst misses
- 🧹 Filtering to NBA–vs–NBA only (no exhibitions)
- 🌌 (Optional) Script / total-error analysis


------

## 🧩 1. Environment & Setup


In [ ]:
# Universal QEPC header for this notebook

import os
import sys
from pathlib import Path

try:
    from notebook_context import *  # try direct import first
except ModuleNotFoundError:
    cwd = Path.cwd()
    candidate_roots = [cwd, cwd.parent, cwd.parent.parent]

    found_root = None
    for root in candidate_roots:
        if (root / "notebook_context.py").exists():
            found_root = root
            break

    if found_root is None:
        raise ModuleNotFoundError(
            f"Could not find notebook_context.py from {cwd}. "
            "Try opening this notebook from inside your qepc_project folder."
        )

    sys.path.insert(0, str(found_root))
    os.chdir(found_root)
    from notebook_context import *

# Fallback for project_root if notebook_context didn't define it
try:
    project_root
except NameError:
    project_root = Path.cwd()

print("Project root:", project_root)


-----

## 📅 2. Backtest Date Range & Filters


In [ ]:
# 2. Backtest Date Range & Filters

# You can change these when you want to test a different window.
# Example choices:
#   "2025-10-21" to today  -> this 2025-26 season so far
#   "2024-10-21" to today  -> last season + this season (if data exists)

from datetime import datetime

start_date = "2025-10-01"  # <-- adjust if needed
end_date = datetime.today().strftime("%Y-%m-%d")

print(f"Backtest date range: {start_date} → {end_date}")


-----

## 🧮 3. Run Season Backtest


In [ ]:
# 3. Run Season Backtest

print(f"Running QEPC backtest from {start_date} to {end_date}...\n")

backtest_long = run_season_backtest(
    start_date=start_date,
    end_date=end_date,
    # add num_trials=... here if your function supports it
)

print("\nFirst 5 rows of backtest results:")
display(backtest_long.head())

print("\nColumns in backtest_long:")
print(backtest_long.columns.tolist())


----

## 🏁 4. Global Summary (All Games in Range)


In [ ]:
# 4. Global Summary (All Games in Range)

total_games = len(backtest_long)
accuracy = backtest_long["Correct_Pick"].mean() * 100
mae_spread = backtest_long["Spread_Error"].abs().mean()

print("========================================")
print("🏆 CUMULATIVE BACKTEST COMPLETE (RAW)")
print("========================================")
print(f"🏀 Games Simulated: {total_games}")
print(f"✅ Overall Accuracy: {accuracy:.2f}%")
print(f"🎯 Avg Spread Error: {mae_spread:.2f} points")


----

## 🧹 5. Clean NBA-Only View (Filter Out Exhibitions)


In [ ]:
# 5. Clean NBA-Only View (Filter Out Exhibitions)

NBA_TEAMS = [
    "Atlanta Hawks", "Boston Celtics", "Brooklyn Nets", "Charlotte Hornets",
    "Chicago Bulls", "Cleveland Cavaliers", "Dallas Mavericks", "Denver Nuggets",
    "Detroit Pistons", "Golden State Warriors", "Houston Rockets", "Indiana Pacers",
    "Los Angeles Clippers", "Los Angeles Lakers", "Memphis Grizzlies", "Miami Heat",
    "Milwaukee Bucks", "Minnesota Timberwolves", "New Orleans Pelicans", "New York Knicks",
    "Oklahoma City Thunder", "Orlando Magic", "Philadelphia 76ers", "Phoenix Suns",
    "Portland Trail Blazers", "Sacramento Kings", "San Antonio Spurs", "Toronto Raptors",
    "Utah Jazz", "Washington Wizards",
]

backtest_clean = backtest_long[
    backtest_long["Away Team"].isin(NBA_TEAMS)
    & backtest_long["Home Team"].isin(NBA_TEAMS)
].copy()

print(f"Original games: {len(backtest_long)}, After NBA-only filter: {len(backtest_clean)}")

acc_clean = backtest_clean["Correct_Pick"].mean() * 100
mae_clean = backtest_clean["Spread_Error"].abs().mean()

print("========================================")
print("🏆 CLEAN NBA-ONLY BACKTEST")
print("========================================")
print(f"🏀 Games Simulated: {len(backtest_clean)}")
print(f"✅ Overall Accuracy: {acc_clean:.2f}%")
print(f"🎯 Avg Spread Error: {mae_clean:.2f} points")


-----

## 🧨 6. Biggest Misses (Spread Error Analysis)


In [ ]:
# 6. Biggest Misses (Spread Error Analysis)

worst = backtest_clean.reindex(
    backtest_clean["Spread_Error"].abs().sort_values(ascending=False).index
)

cols = [
    "Date",
    "Away Team", "Home Team",
    "Expected_Spread", "Actual_Spread", "Spread_Error",
    "Away_Win_Prob", "Home_Win_Prob",
    "Sim_Away_Score", "Actual_Away_Score",
    "Sim_Home_Score", "Actual_Home_Score",
    "Correct_Pick",
]
cols = [c for c in cols if c in worst.columns]

print("Top 15 biggest spread misses (NBA vs NBA only):")
display(worst[cols].head(15))


----

## 📦 7. Spread Error Buckets (How Often & How Bad)


In [ ]:
# 7. Spread Error Buckets (How Often & How Bad)

abs_err = backtest_clean["Spread_Error"].abs()

bins = [0, 5, 10, 15, 20, np.inf]
labels = ["0–5", "5–10", "10–15", "15–20", "20+"]

backtest_clean["Spread_Error_Bucket"] = pd.cut(
    abs_err, bins=bins, labels=labels, right=False
)

bucket_counts = backtest_clean["Spread_Error_Bucket"].value_counts().sort_index()
bucket_pct = (bucket_counts / len(backtest_clean) * 100).round(2)

spread_bucket_summary = pd.DataFrame({
    "Games": bucket_counts,
    "Percent": bucket_pct
})

print("Spread error distribution (absolute error in points):")
display(spread_bucket_summary)


----

## 🌌 8. Optional: Total Error & Script Classification (GRIND / BASE / CHAOS)


-----

In [ ]:
# 8. Optional: Total Error & Script Classification (GRIND / BASE / CHAOS)

df = backtest_clean.copy()

# 8.1 Compute simulated and actual totals
df["Sim_Total"] = df["Sim_Home_Score"] + df["Sim_Away_Score"]
df["Actual_Total"] = df["Actual_Home_Score"] + df["Actual_Away_Score"]

# Drop games where we effectively didn't simulate (Sim_Total == 0)
before = len(df)
df = df[df["Sim_Total"] > 0]
after = len(df)
print(f"Dropped {before - after} games with Sim_Total == 0.")
print(f"Remaining games for script analysis: {after}")

# 8.2 Total error
df["Total_Error"] = df["Actual_Total"] - df["Sim_Total"]

# Thresholds for GRIND / BASE / CHAOS (can be tuned)
grind_thresh = -15   # 15+ pts under model total → GRIND
chaos_thresh = 15    # 15+ pts over model total → CHAOS

def classify_script(row):
    if row["Total_Error"] <= grind_thresh:
        return "GRIND"
    elif row["Total_Error"] >= chaos_thresh:
        return "CHAOS"
    else:
        return "BASE"

df["Script_ExPost"] = df.apply(classify_script, axis=1)

print("\nSample of script labels:")
display(
    df[["Date", "Away Team", "Home Team", "Sim_Total", "Actual_Total", "Total_Error", "Script_ExPost"]]
    .head()
)

# 8.3 Script-level summary
script_summary = df.groupby("Script_ExPost").agg(
    Games=("Total_Error", "count"),
    Avg_Total_Error=("Total_Error", "mean"),
    Avg_Abs_Total_Error=("Total_Error", lambda x: x.abs().mean()),
    Avg_Spread_Error=("Spread_Error", "mean"),
    Avg_Abs_Spread_Error=("Spread_Error", lambda x: x.abs().mean()),
)

script_summary["Percent"] = (script_summary["Games"] / len(df) * 100).round(2)

print("\nEx-post script distribution (based on totals):")
display(script_summary)
